# Model Development - Linear Regression, Gradient Boosting & LightGBM

<b> NOTE: </b>Following the notebooks https://github.com/McGill-MMA-EnterpriseAnalytics/Customer-Aquisition/blob/main/Preprocessing%20(v0.0).ipynb and https://github.com/McGill-MMA-EnterpriseAnalytics/Customer-Aquisition/blob/DataScientist/INSY_695_Feature_Selection.ipynb, the same steps were applied to pre-process the data, and the features obtained from the feature selection process were used for the model development. 

In [70]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as ltb
import sklearn.metrics as metrics

In [20]:
df_raw = pd.read_csv("CustomerAcquisition.csv")

In [21]:
num_cols, cate_cols, pre_dum_cols = [], [], []
for c in df_raw.columns:
    uni_len = len(set(df_raw[c]))
    dtype = df_raw[c].dtypes

    if dtype == object:
        cate_cols.append(c)
    else: # dtype == float
        if uni_len == 2:
            pre_dum_cols.append(c)
        elif uni_len < 10:
            num_cols.append(c)
            print("Number of unique values: %d \t Datatype: %s \t Column name: %s" %(uni_len, dtype, c))
        else:
            num_cols.append(c)

Number of unique values: 6 	 Datatype: float64 	 Column name: unit_sales(in millions)
Number of unique values: 6 	 Datatype: float64 	 Column name: total_children
Number of unique values: 5 	 Datatype: float64 	 Column name: avg_cars_at home(approx)
Number of unique values: 6 	 Datatype: float64 	 Column name: num_children_at_home
Number of unique values: 5 	 Datatype: float64 	 Column name: avg_cars_at home(approx).1


In [22]:
warnings.filterwarnings("ignore")

isolation_forest = IsolationForest(contamination = 0.05, max_samples = 100, random_state = 0)

# DataFrame to store the outlier predictions
outlier_preds = pd.DataFrame()

# Iterate all numerical columns
for col in num_cols:
    # Fit the Isolation Forest to the current column
    isolation_forest.fit(df_raw[[col]])
    
    # Get the outlier predictions for the current column
    outlier_pred = isolation_forest.predict(df_raw[[col]])
    
    # Add the outlier predictions as a new column in the outlier_preds DataFrame
    outlier_preds[col] = outlier_pred

outlier_preds

,store_sales(in millions),store_cost(in millions),unit_sales(in millions),total_children,avg_cars_at home(approx),num_children_at_home,avg_cars_at home(approx).1,SRP,gross_weight,net_weight,units_per_case,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,cost
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60423,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1
60424,1,-1,-1,1,1,1,1,1,1,1,-1,1,1,1,1,1
60425,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
60426,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1


In [23]:
df_woOut = df_raw.iloc[outlier_pred == 1]
df_woOut

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,...,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4.0,Bag Stuffers,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",126.62
1,Breakfast Foods,Frozen Foods,Food,5.52,2.5944,3.0,Cash Register Lottery,USA,M,M,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",59.86
2,Breakfast Foods,Frozen Foods,Food,3.68,1.3616,2.0,High Roller Savings,USA,S,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",84.16
3,Breakfast Foods,Frozen Foods,Food,3.68,1.1776,2.0,Cash Register Lottery,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,In-Store Coupon,95.78
6,Breakfast Foods,Frozen Foods,Food,5.44,2.5568,4.0,Cash Register Lottery,USA,S,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,In-Store Coupon,95.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60423,Specialty,Carousel,Non-Consumable,2.76,1.3248,1.0,You Save Days,USA,M,F,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,In-Store Coupon,95.25
60424,Specialty,Carousel,Non-Consumable,1.60,0.4960,1.0,Price Cutters,USA,S,F,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,Sunday Paper,69.42
60425,Specialty,Carousel,Non-Consumable,5.52,2.5392,2.0,Weekend Markdown,USA,M,M,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,"Sunday Paper, Radio, TV",67.51
60426,Specialty,Carousel,Non-Consumable,8.28,2.5668,3.0,Sales Days,Canada,S,M,...,27463.0,4193.0,2795.0,1.0,0.0,0.0,0.0,1.0,Sunday Paper,132.88


# Feature Engineering

In [24]:
warnings.filterwarnings("ignore")

df_woOut['sales_to_cost_ratio'] = df_woOut['store_sales(in millions)'] / df_woOut['store_cost(in millions)']
df_woOut['frozen_to_meat_ratio'] = df_woOut['frozen_sqft'] / df_woOut['meat_sqft']
df_woOut['meat_to_total_ratio'] = df_woOut['meat_sqft'] / df_woOut['store_sqft']

In [25]:
df_woOut = df_woOut.drop(['store_sales(in millions)', 'store_cost(in millions)', 'frozen_sqft', 'meat_sqft', 'grocery_sqft', 'store_sqft',
                          'gross_weight', 'avg_cars_at home(approx).1', 'grocery_sqft'], axis = 1)

In [26]:
num_cols, cate_cols, pre_dum_cols = [], [], []
for c in df_woOut.columns:
    uni_len = len(set(df_woOut[c]))
    dtype = df_woOut[c].dtypes

    if dtype == object:
        cate_cols.append(c)
    else: # dtype == float
        if uni_len == 2:
            pre_dum_cols.append(c)
        elif uni_len < 10:
            num_cols.append(c)
            print("Number of unique values: %d \t Datatype: %s \t Column name: %s" %(uni_len, dtype, c))
        else:
            num_cols.append(c)

Number of unique values: 6 	 Datatype: float64 	 Column name: unit_sales(in millions)
Number of unique values: 6 	 Datatype: float64 	 Column name: total_children
Number of unique values: 5 	 Datatype: float64 	 Column name: avg_cars_at home(approx)
Number of unique values: 6 	 Datatype: float64 	 Column name: num_children_at_home


In [27]:
cols = [c for c in num_cols if c != 'cost'] # exclude target

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
MinMax_scaled = min_max_scaler.fit_transform(df_woOut[cols])

MinMax_scaled

array([[ 0.2       , -0.6       , -0.5       , ...,  0.05405405,
         0.11000332,  0.26845737],
       [-0.2       , -1.        ,  1.        , ..., -0.80851064,
         0.11000332,  0.26845737],
       [-0.6       ,  0.6       , -0.5       , ...,  0.05405405,
         0.11000332,  0.26845737],
       ...,
       [-0.6       , -0.6       ,  0.5       , ..., -0.73913043,
        -0.4091355 ,  0.19257961],
       [-0.2       , -0.2       ,  1.        , ...,  0.83870968,
         0.64177236, -0.98319777],
       [ 0.2       ,  0.2       , -0.5       , ..., -0.73913043,
         0.64177236, -0.98319777]])

In [28]:
cols = [c for c in num_cols if c != 'cost'] # exclude target

std_scaler = StandardScaler()
std_scaled = std_scaler.fit_transform(df_woOut[cols])
std_scaled

array([[ 1.09602519, -1.03128914, -1.08330242, ...,  0.39259738,
        -0.01604716,  0.18363349],
       [-0.11019699, -1.70201487,  1.61780374, ..., -1.11761199,
        -0.01604716,  0.18363349],
       [-1.31641916,  0.98088805, -1.08330242, ...,  0.39259738,
        -0.01604716,  0.18363349],
       ...,
       [-1.31641916, -1.03128914,  0.71743502, ..., -0.99613863,
        -0.97003583,  0.05803928],
       [-0.11019699, -0.36056341,  1.61780374, ...,  1.76640074,
         0.96115125, -1.88812817],
       [ 1.09602519,  0.31016232, -1.08330242, ..., -0.99613863,
         0.96115125, -1.88812817]])

In [29]:
#df = pd.DataFrame(std_scaled, columns = [c for c in num_cols if c != 'cost'])
df = pd.DataFrame(MinMax_scaled, columns = [c for c in num_cols if c != 'cost'])

# Concatenate standardized/normalized numerical columns with the others
df = pd.concat([df.reset_index(drop=True), 
                df_woOut[cate_cols + pre_dum_cols].reset_index(drop=True)], axis=1)

## Dropping irrelevant attributes

In [30]:
df = df.drop(['food_category', 'food_department', 'brand_name', 'store_city', 'promotion_name'], axis = 1)


## Dummify categorical variables

In [31]:
from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit_transform(df['media_type'].tolist())

# to decode values 
# label_encoder.inverse_transform([1, 1, 0])

df['media_type_labeled'] = label_encoder.fit_transform(df['media_type'].tolist())

In [32]:
# Dummify categorical variables except for 'media type'
df = pd.get_dummies(df[[c for c in df.columns if c != 'media_type']], drop_first = True)

In [33]:
df

,unit_sales(in millions),total_children,avg_cars_at home(approx),num_children_at_home,SRP,net_weight,units_per_case,sales_to_cost_ratio,frozen_to_meat_ratio,meat_to_total_ratio,...,store_type_Supermarket,store_state_CA,store_state_DF,store_state_Guerrero,store_state_Jalisco,store_state_OR,store_state_Veracruz,store_state_WA,store_state_Yucatan,store_state_Zacatecas
0,0.2,-0.6,-0.5,-0.6,-0.229885,0.650704,-0.085714,0.054054,0.110003,0.268457,...,0,0,0,0,0,1,0,0,0,0
1,-0.2,-1.0,1.0,-1.0,-0.229885,0.650704,-0.085714,-0.808511,0.110003,0.268457,...,0,0,0,0,0,1,0,0,0,0
2,-0.6,0.6,-0.5,-1.0,-0.229885,0.650704,-0.085714,0.054054,0.110003,0.268457,...,0,0,0,0,0,1,0,0,0,0
3,-0.6,-0.2,0.0,-0.2,-0.229885,0.650704,-0.085714,0.687500,0.110003,0.268457,...,0,0,0,0,0,1,0,0,0,0
4,0.2,0.6,0.0,-1.0,-0.505747,-0.767887,0.600000,-0.808511,0.110003,0.268457,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57418,-1.0,-0.6,-0.5,-0.6,0.298851,0.752113,0.428571,-0.875000,-0.409135,0.192580,...,0,1,0,0,0,0,0,0,0,0
57419,-1.0,-0.2,0.0,-1.0,-0.367816,0.380282,1.000000,0.838710,-0.409135,0.192580,...,0,1,0,0,0,0,0,0,0,0
57420,-0.6,-0.6,0.5,-1.0,0.298851,0.752113,0.428571,-0.739130,-0.409135,0.192580,...,0,1,0,0,0,0,0,0,0,0
57421,-0.2,-0.2,1.0,-1.0,0.298851,0.752113,0.428571,0.838710,0.641772,-0.983198,...,0,0,0,0,0,0,0,0,0,0


<b>NOTE:</b> The features selected below are based on the feature selection done in notebook https://github.com/McGill-MMA-EnterpriseAnalytics/Customer-Aquisition/blob/DataScientist/INSY_695_Feature_Selection.ipynb by using a RandomForestRegressor to obtain the significance of the features.

In [34]:
X = df[['media_type_labeled', 'SRP', 'net_weight', 'units_per_case', 'sales_to_cost_ratio', 'total_children', 'avg_cars_at home(approx)', 'meat_to_total_ratio',
              'num_children_at_home', 'unit_sales(in millions)', 'gender_M', 'houseowner_Y']]
y = df_woOut['cost']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## Linear Regression

A linear regression model was developed to obtain the coefficients, which in turn will allow the business analysts to better understand the impact that each selected feature has on the cost of acquiring a customer.

In [68]:
# Run linear regression
lm = LinearRegression()
model = lm.fit(X_train,y_train)

# Using the model to predict the results based on the test dataset
y_test_pred = model.predict(X_test)

# Calculate the mean squared error of the prediction
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE = "+str(mse)) 

# Using the model to predict the results based on the training dataset
y_train_pred = model.predict(X_train)

# Calculate the mean squared error of the prediction
mse2 = mean_squared_error(y_train, y_train_pred)
print("Train MSE = "+str(mse2))

print("Intercept:",model.intercept_)

col_names = X.columns.values.tolist()

df_coeff = pd.DataFrame({"Feature":col_names,
                        "Coefficient":model.coef_})

display(df_coeff)

Test MSE = 823.1718741003523
Train MSE = 820.3323497445925
Intercept: 99.92598044578328


,Feature,Coefficient
0,media_type_labeled,-0.071215
1,SRP,-0.039037
2,net_weight,0.137395
3,units_per_case,-0.154941
4,sales_to_cost_ratio,-0.160111
5,total_children,-0.144315
6,avg_cars_at home(approx),0.933811
7,meat_to_total_ratio,-2.656683
8,num_children_at_home,-0.091293
9,unit_sales(in millions),-1.662274


Other models were tried out to get a better predictive performance:

## Gradient Boosting 

In [60]:
# Run gradient boosting 
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Using the model to predict the results based on the test dataset
y_test_pred = model.predict(X_test)


# Calculate the mean squared error of the prediction
print("Test MSE = ", metrics.mean_squared_error(y_test, y_test_pred))

# Using the model to predict the results based on the training dataset
y_train_pred = model.predict(X_train)


# Calculate the mean squared error of the prediction
print("Train MSE = ", metrics.mean_squared_error(y_train, y_train_pred))

Test MSE =  612.8834840478977
Train MSE =  606.1166131917934


## LightGBM

In [58]:
# Run LGBM Regressor
model = ltb.LGBMRegressor()
model.fit(X_train, y_train)

# Using the model to predict the results based on the test dataset
y_test_pred = model.predict(X_test)

# Calculate the mean squared error of the prediction
print("Test MSE = ", metrics.mean_squared_error(y_test, y_test_pred))

# Using the model to predict the results based on the training dataset
y_train_pred = model.predict(X_train)

# Calculate the mean squared error of the prediction
print("Train MSE = ", metrics.mean_squared_error(y_train, y_train_pred))

Test MSE =  402.9000885733051
Train MSE =  384.1543881006732


From the above experiments, we can deduce that the LightGBM model yields the best results.